TODO uncomment line below before p2j.

In [ ]:
%matplotlib inline
from matplotlib import style

In [ ]:
style.use('dark_background')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

Reflect Tables into SQLAlchemy ORM<br>
create engine to hawaii.sqlite

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)
conn = engine.connect()

reflect an existing database into a new model

In [ ]:
Base = automap_base()

reflect the tables

In [ ]:
Base.prepare(engine, reflect=True)

View all of the classes that automap found

In [ ]:
Base.classes.keys()

Save references to each table

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station

Create our session (link) from Python to the DB

In [ ]:
session = Session(engine)

Exploratory Precipitation Analysis<br>
Find the most recent date in the data set.

In [ ]:
first_date = session.query(
    Measurement.date
).order_by(
    Measurement.date
).first()

In [ ]:
last_date = session.query(
    Measurement.date
).order_by(
    Measurement.date.desc()
).first()

In [ ]:
last_date = dt.datetime.strptime(last_date[0], '%Y-%m-%d')

Design a query to retrieve the last 12 months of precipitation data and plot the results.<br>
Starting from the most recent data point in the database.<br>
Calculate the date one year from the last date in data set.

In [ ]:
year_ago = last_date - dt.timedelta(days=365)

Perform a query to retrieve the data and precipitation scores

In [ ]:
precip = session.query(
    Measurement.date,
    Measurement.prcp
).filter(
    Measurement.date > year_ago
).order_by(Measurement.date).all()

Save the query results as a Pandas DataFrame and set the index to the date column<br>
Sort the dataframe by date

In [ ]:
precip_df = pd.DataFrame(
    precip,
    columns=['date', 'prcp']
).set_index('date').sort_index(ascending=True)

Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
fig, ax = plt.subplots()
plt.rcParams.update({'font.size': 16})

In [ ]:
precip_df.plot(
    figsize=(15, 12),
    title='Precipitation Data for August 2016 to August 2017',
    xlabel='Date',
    ylabel='Precipitation',
    fontsize=14,
    rot=45
)

In [ ]:
plt.legend(['Precipitation'])
plt.tight_layout()

In [ ]:
plt.savefig(r'Resources/precip.png', dpi=300)

Use Pandas to calculate the summary statistics for the precipitation data

In [ ]:
precip_df.describe()

Exploratory Station Analysis<br>
Design a query to calculate the total number stations in the dataset

In [ ]:
Stations = session.query(Station)
num_stations = Stations.count()

In [ ]:
print(
    f'There are {num_stations} stations in this data.'
)

Design a query to find the most active stations<br>
(i.e. what stations have the most rows?)<br>
List the stations and the counts in descending order.

In [ ]:
active_stations = session.query(
    Measurement.station, func.count(Measurement.station)
).group_by(
    Measurement.station
).order_by(
    func.count(Measurement.station).desc()
).all()

In [ ]:
for station, count in active_stations:
    print(f'Station {station} has {count} measurements.')

Using the most active station id from the previous query,<br>
calculate the lowest, highest, and average temperature.

In [ ]:
station = active_stations[0][0]

In [ ]:
temps = session.query(
    Measurement.station, Measurement.date, Measurement.tobs
).filter(
    Measurement.station == station
).filter(
    Measurement.date > year_ago
).order_by(
    Measurement.date
).all()

In [ ]:
temp_df = pd.DataFrame(
    temps,
    columns=['station', 'date', 'tobs']
)

In [ ]:
low, high, mean = temp_df['tobs'].min(), temp_df['tobs'].max(), temp_df['tobs'].mean()

In [ ]:
print(
    f'Low: {low}\n'
    f'High: {high}\n'
    f'Mean: {mean}'
)

Using the most active station id<br>
Query the last 12 months of temperature observation data for this station<br>
and plot the results as a histogram

In [ ]:
fig, ax = plt.subplots()

In [ ]:
temp_df.plot(
    kind='hist',
    figsize=(15, 12),
    title='Observed Temperatures for August 2016 to August 2017',
    fontsize=16,
    width=2.3
)

In [ ]:
plt.xlabel('Temperature')
plt.ylabel('Observations')
plt.legend(['Observed Temps'])

In [ ]:
plt.tight_layout()
plt.savefig(r'Resources/temp_dist.png', dpi=300)

Close Session

In [ ]:
session.close()